In [1]:
import cv2
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pydicom
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook

In [53]:
train_data_dir = 'H:/Datasets/'
data_dir = "D:/Datasets/rsna/"
meta_data_train_dir = data_dir + "metadata/train/"
meta_data_test_dir = data_dir + "metadata/valid/"

In [71]:
train_mtd = pd.read_csv(os.path.join(data_dir,"train_metadata.csv"))

In [72]:
valid_fold1 = train_mtd[train_mtd['fold']==4].reset_index(drop=True)
train_1_3 = train_mtd[train_mtd['fold']!=4].reset_index(drop=True)

In [73]:
train_df_13 = train_1_3.drop_duplicates(subset=['StudyInstanceUID'],inplace=True)
valid_df0 = valid_fold0.drop_duplicates(subset=['StudyInstanceUID'],inplace=True)

In [74]:
train_1_3.to_csv(os.path.join(data_dir, meta_data_train_dir,"train_minus_fold4.csv"))
valid_fold0.to_csv(os.path.join(data_dir, meta_data_test_dir,"valid_fold4.csv"))

In [10]:
valid_fold0_u = valid_fold0["StudyInstanceUID"].drop

In [13]:
len(valid_fold0_u)

4373

In [25]:
train_1_3 = train_mtd[train_mtd['fold']!=0].reset_index(drop=True)

In [26]:
len(train_1_3)

601443

In [27]:
train_df_13 = train_1_3.drop_duplicates(subset=['StudyInstanceUID'],inplace=True)

In [31]:
len(train_1_3['StudyInstanceUID'].unique())

17371

,image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,ImagePositionPatient_2,PatientID,StudyInstanceUID,WindowCenter,WindowWidth,fold
0,ID_1b804da9a,0,0,0,0,0,0,30.645344,ID_fffc2bd6,ID_dbf89aa342,30,80,3
28,ID_47cb66839,0,0,0,0,0,0,-20.603000,ID_fff502d5,ID_f10de2d018,40,150,2
64,ID_37292c56f,0,0,0,0,0,0,-2.500000,ID_fff140ff,ID_33b54f267c,35,135,4
100,ID_9328f5127,0,0,0,0,0,0,874.299988,ID_ffedaf23,ID_76a555900b,36,80,1
137,ID_d8e20e949,0,0,0,0,0,0,47.726818,ID_ffeabb7c,ID_c73e7b76c3,30,80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601263,ID_b5429c663,0,0,0,0,0,0,82.599976,ID_000e5623,ID_bd2fd686ee,36,80,1
601296,ID_31b14de96,0,0,0,0,0,0,32.955000,ID_00086119,ID_fdde2979b0,30,80,1
601336,ID_c6f9f68c9,0,0,0,0,0,0,38.171000,ID_0006d192,ID_25690b4725,40,150,3
601376,ID_138d275c8,0,0,0,0,0,0,71.900024,ID_00054f3f,ID_8a449ae31b,36,80,1


In [3]:
dropped_cols = ['BitsAllocated', 'BitsStored',
       'Columns', 'HighBit', 'ImageOrientationPatient_0',
       'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
       'ImageOrientationPatient_3', 'ImageOrientationPatient_4',
       'ImageOrientationPatient_5', 'ImagePositionPatient_0',
       'ImagePositionPatient_1', 'Modality',
       'PhotometricInterpretation', 'PixelRepresentation',
       'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope',
       'Rows', 'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID',
       'StudyID',]

## Prepare the labels & metadata

In [4]:
def get_metadata(image_dir):

    labels = [
        'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
        'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
        'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
        'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
        'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
        'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
        'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
        'WindowCenter', 'WindowWidth', 'Image',
    ]

    data = {l: [] for l in labels}

    for image in tqdm_notebook(os.listdir(image_dir)):
        data["Image"].append(image[:-4])
        try:
            ds = pydicom.dcmread(os.path.join(image_dir, image))

            for metadata in ds.dir():
                if metadata != "PixelData":
                    metadata_values = getattr(ds, metadata)
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                        for i, v in enumerate(metadata_values):
                            data[f"{metadata}_{i}"].append(v)
                    else:
                        if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                            data[metadata].append(metadata_values[0])
                        else:
                            data[metadata].append(metadata_values)
        except:
            print(image)
            continue
    return pd.DataFrame(data).set_index("Image")

In [5]:
# Generate metadata dataframes
train_metadata = get_metadata(os.path.join(data_dir, "stage_2_train"))
test_metadata = get_metadata(os.path.join(data_dir, "stage_2_test"))


<ipython-input-4-32eeaeee22c6>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image in tqdm_notebook(os.listdir(image_dir)):


  0%|          | 0/752803 [00:00<?, ?it/s]

  0%|          | 0/121232 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Datasets/rsna1/gz//train_metadata.parquet.gzip'

In [8]:
train_metadata.to_parquet(f'{meta_data_dir}train_metadata.parquet.gzip', compression='gzip')
test_metadata.to_parquet(f'{meta_data_dir}test_metadata.parquet.gzip', compression='gzip')

### Note: 'ImagePositionPatient_2' is z-axis

In [9]:
train_df = pd.read_csv(data_dir + "stage_2_train.csv").drop_duplicates()
train_df['image'] = train_df["ID"].str.slice(stop=12)
train_df['Diagnosis'] = train_df['ID'].str.slice(start=13)
train_labels = train_df.pivot(index="image", 
                              columns="Diagnosis", 
                              values="Label")

In [10]:
test_df = pd.read_csv(data_dir + "stage_2_sample_submission.csv").drop_duplicates()
test_df['image'] = test_df["ID"].str.slice(stop=12)

In [11]:
train = pd.concat([train_labels["any"],
                   train_labels["epidural"], train_labels["intraparenchymal"],
                   train_labels["intraventricular"], train_labels["subarachnoid"],
                   train_labels["subdural"]], 1).reset_index()

<ipython-input-11-6746f4d1ff98>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  train = pd.concat([train_labels["any"],


In [12]:
test = test_df["image"].drop_duplicates()

In [13]:
train_parq = pd.read_parquet(meta_data_dir + "train_metadata.parquet.gzip")
train_parq["image"] = train_parq.index
test_parq = pd.read_parquet(meta_data_dir + "test_metadata.parquet.gzip")
test_parq["image"] = test_parq.index

In [14]:
merged_train = pd.merge(train, train_parq, how="inner", on="image")
merged_test = pd.merge(test, test_parq, how="inner", on="image")
merged_train.drop(columns=dropped_cols, inplace=True)
merged_test.drop(columns=dropped_cols, inplace=True)

del train, test, train_parq, test_parq
gc.collect()

merged_train = merged_train.groupby(["StudyInstanceUID"]) \
    .apply(lambda x: x.sort_values(["ImagePositionPatient_2"], 
                                   ascending = True))\
    .reset_index(drop=True)

merged_test = merged_test.groupby(["StudyInstanceUID"]) \
    .apply(lambda x: x.sort_values(["ImagePositionPatient_2"], 
                                   ascending = True)) \
    .reset_index(drop=True)

merged_train.to_csv(data_dir + "train_metadata.csv", index=False)
merged_test.to_csv(data_dir + "test_metadata.csv", index=False)

In [15]:
merged_train

,image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,ImagePositionPatient_2,PatientID,StudyInstanceUID,WindowCenter,WindowWidth
0,ID_3a422b8d7,0,0,0,0,0,0,28.2225,ID_e0d2de32,ID_00047d6503,30.0,80.0
1,ID_490b10d5a,0,0,0,0,0,0,33.2225,ID_e0d2de32,ID_00047d6503,30.0,80.0
2,ID_be2a0ca1c,0,0,0,0,0,0,38.2225,ID_e0d2de32,ID_00047d6503,30.0,80.0
3,ID_af42e31f3,0,0,0,0,0,0,43.2225,ID_e0d2de32,ID_00047d6503,30.0,80.0
4,ID_3131664ab,0,0,0,0,0,0,48.2225,ID_e0d2de32,ID_00047d6503,30.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...
752798,ID_7b0c5edc0,0,0,0,0,0,0,1310.0500,ID_822276a2,ID_fffdba8d7b,40.0,80.0
752799,ID_e8e195f90,0,0,0,0,0,0,1315.0500,ID_822276a2,ID_fffdba8d7b,40.0,80.0
752800,ID_d64ef9ea6,0,0,0,0,0,0,1320.0500,ID_822276a2,ID_fffdba8d7b,40.0,80.0
752801,ID_5838a09b1,0,0,0,0,0,0,1325.0500,ID_822276a2,ID_fffdba8d7b,40.0,80.0


## Split by StudyInstanceUID

In [16]:
study_id_counts = merged_train.groupby(["StudyInstanceUID"])["any"].sum()
study_ids, slice_counts = study_id_counts.index.values, study_id_counts.values

In [17]:
binarized_slice_counts = slice_counts > 0.

In [19]:
skf = StratifiedKFold(n_splits=5, random_state=2709,shuffle=True)

In [20]:
folds = {}
for i, (train_idx, val_idx) in enumerate(skf.split(study_ids, binarized_slice_counts)):
    folds[i] = (study_ids[train_idx], study_ids[val_idx])

    np.save(data_dir + "train_fold" + str(i) + ".npy", study_ids[train_idx])
    np.save(data_dir + "valid_fold" + str(i) + ".npy", study_ids[val_idx])

In [21]:
for fold in range(5):
    print("\n================FOLD {}================".format(str(fold)))
    
    for col in [
        "any", 
        "intraparenchymal", "intraventricular", 
        "subarachnoid", "subdural", "epidural"
    ]:
        train_df = merged_train[merged_train["StudyInstanceUID"].isin(folds[fold][0])]
        num_train_strat_sid = train_df[col].sum()
        '''
        print(col,
              "train samples: {:.1f} ===".format(num_train_strat_sid),
              "alpha: {:.3f}".format(compute_alpha(
                  num_train_strat_sid / train_df['epidural'].sum())
                                    )
             )'''


================FOLD 0================

================FOLD 1================

================FOLD 2================

================FOLD 3================

================FOLD 4================


## Split by PatientID

In [22]:
patient_id_counts = merged_train.groupby(["PatientID"])["any"].sum()
patient_ids, slice_counts = patient_id_counts.index.values, patient_id_counts.values

In [23]:
patient_ids

array(['ID_0002cd41', 'ID_00054f3f', 'ID_0006d192', ..., 'ID_fff140ff',
       'ID_fff502d5', 'ID_fffc2bd6'], dtype=object)

In [24]:
slice_counts

array([ 0,  0,  0, ...,  6, 11,  0], dtype=int64)

In [25]:
binarized_slice_counts = slice_counts > 0.

In [26]:
binarized_slice_counts.shape

(18938,)

In [27]:
patient_ids.shape

(18938,)

In [17]:
import pandas as pd
import numpy as np
#data = np.load("D:/Datasets/rsna/train_patients_fold0.npy", allow_pickle=True)
pdf = pd.read_csv("D:/Datasets/rsna/train_metadata.csv")
len(pdf["StudyInstanceUID"].unique())

21744

In [28]:
np.sum(binarized_slice_counts) / 17079

0.4480355992739622

In [29]:
skf = StratifiedKFold(n_splits=5, random_state=2709,shuffle=True)

folds = {}
for i, (train_idx, val_idx) in enumerate(skf.split(patient_ids, binarized_slice_counts)):
    folds[i] = (patient_ids[train_idx], patient_ids[val_idx])
    np.save(data_dir + "train_patients_fold" + str(i) + ".npy", patient_ids[train_idx])
    np.save(data_dir + "valid_patients_fold" + str(i) + ".npy", patient_ids[val_idx])

In [30]:
for fold in range(5):
    print("\n================FOLD {}================".format(str(fold)))
    
    for col in [
        "any", 
        "intraparenchymal", "intraventricular", 
        "subarachnoid", "subdural", "epidural"
    ]:
        train_df = merged_train[merged_train["PatientID"].isin(folds[fold][0])]
        num_train_strat_sid = train_df[col].sum()
        print(col,
              "train samples: {:.1f} ===".format(num_train_strat_sid),
             )


================FOLD 0================
any train samples: 85704.0 ===
intraparenchymal train samples: 28666.0 ===
intraventricular train samples: 21108.0 ===
subarachnoid train samples: 27832.0 ===
subdural train samples: 37234.0 ===
epidural train samples: 2532.0 ===

================FOLD 1================
any train samples: 86417.0 ===
intraparenchymal train samples: 28671.0 ===
intraventricular train samples: 20919.0 ===
subarachnoid train samples: 28654.0 ===
subdural train samples: 37891.0 ===
epidural train samples: 2479.0 ===

================FOLD 2================
any train samples: 86339.0 ===
intraparenchymal train samples: 29199.0 ===
intraventricular train samples: 20843.0 ===
subarachnoid train samples: 28484.0 ===
subdural train samples: 37769.0 ===
epidural train samples: 2528.0 ===

================FOLD 3================
any train samples: 86735.0 ===
intraparenchymal train samples: 29082.0 ===
intraventricular train samples: 21161.0 ===
subarachnoid train samples: 287

In [31]:
train_meta = pd.read_csv(os.path.join(data_dir,"train_metadata.csv")) 
#train_meta = pd.read_csv("/mnt/DATA/rsna/train_metadata.csv")
test_meta = pd.read_csv(os.path.join(data_dir,"test_metadata.csv"))
# check patient id overlap
print(np.intersect1d(train_meta["PatientID"].unique(),
               test_meta["PatientID"].unique()).shape)
# check train patient id
print(train_meta["PatientID"].unique().shape)
# check test patient id
print(test_meta["PatientID"].unique().shape)

(0,)
(18938,)
(3518,)


In [32]:
overlap_patient_ids = np.intersect1d(train_meta["PatientID"].unique(),
                                     test_meta["PatientID"].unique())

In [33]:
test_meta[test_meta["PatientID"].isin(overlap_patient_ids)]

,image,ImagePositionPatient_2,PatientID,StudyInstanceUID,WindowCenter,WindowWidth
